In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_reservation = pl.read_parquet(datadir / "reservation.parquet")
pl_holiday     = pl.read_parquet(datadir / "holiday.parquet")

# 12章 日時
## 12-1 日時型への変換
### Q: 文字列の日時を日時型に変換


#### Awesome

In [ ]:
(
    pl.DataFrame({"datetime_str": ["2023-04-01 10:11:30"]})
    .with_columns(
        # （1） tz-naiveなDatetimeへ変換
        datetime_naive=pl.col("datetime_str")
            .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S"),
        # （2） Dateへ変換
        date=pl.col("datetime_str").str.strptime(pl.Date, "%Y-%m-%d %H:%M:%S"),
        # （3） Timeへ変換
        time=pl.col("datetime_str").str.strptime(pl.Time, "%Y-%m-%d %H:%M:%S"),
        # （4） tz-awareなDatetimeへ変換 (JST)
        datetime_jst=
            pl.col("datetime_str").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S")
            .dt.replace_time_zone("Asia/Tokyo"),
        # （5） tz-awareなDatetimeへ変換 (PST)
        datetime_pst=
            pl.col("datetime_str").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S")
            .dt.replace_time_zone("America/Los_Angeles"),
    )
)

## 12-2 日時型の列を用いた行の抽出
### Q: 指定期間に予約された予約履歴の抽出


#### Awesome

In [ ]:
# reserved_atが指定期間の行を抽出
pl_reservation.filter(
    # reserved_atをdateに変換
    pl.col("reserved_at").dt.date()
    # dateが指定期間かどうかを判定
    .is_between(pl.date(2019, 7, 1), pl.date(2019, 8, 31))
)

## 12-3 日時要素の抽出
### Q: 予約日時の日時要素を抽出


#### Awesome

In [ ]:
(
    pl_reservation
    .select("reserved_at")
    .with_columns(
        # （1） 年を抽出
        reserved_at_year=pl.col("reserved_at").dt.year(),
        # （2） 月を抽出
        reserved_at_month=pl.col("reserved_at").dt.month(),
        # （3） 日を抽出
        reserved_at_day=pl.col("reserved_at").dt.day(),
        # （4） 曜日を抽出 （週の最初を月曜日とした1〜7の数字）
        reserved_at_dayofweek=pl.col("reserved_at").dt.weekday(),
        # （5） 週を抽出 （ISO 8601形式の週番号）
        reserved_at_isoweek=pl.col("reserved_at").dt.week(),
        # （6） 時を抽出
        reserved_at_hour=pl.col("reserved_at").dt.hour(),
        # （7） 分を抽出
        reserved_at_minute=pl.col("reserved_at").dt.minute(),
        # （8） 秒を抽出
        reserved_at_second=pl.col("reserved_at").dt.second(),
        # （9） 年月を抽出
        reserved_at_ym=pl.col("reserved_at").dt.strftime("%Y-%m")
    )
)

## 12-4 日時の丸め処理
### Q: 予約日時の切り捨て


#### Awesome

In [ ]:
(
    pl_reservation
    .select("reserved_at")
    .with_columns(
        # （1） 年に切り捨て
        reserved_at_year=pl.col("reserved_at").dt.truncate("1y"),
        # （2） 四半期に切り捨て
        reserved_at_quarter=pl.col("reserved_at").dt.truncate("3mo"),
        # （3） 月に切り捨て
        reserved_at_month=pl.col("reserved_at").dt.truncate("1mo"),
        # （4） 日に切り捨て
        reserved_at_day=pl.col("reserved_at").dt.truncate("1d"),
        # （5） 時に切り捨て
        reserved_at_hour=pl.col("reserved_at").dt.truncate("1h"),
        # （6） 分に切り捨て
        reserved_at_minute=pl.col("reserved_at").dt.truncate("1m"),
    )
)

## 12-5 日時の加減算
### Q: 予約日時に一定時間を加算


#### Awesome

In [ ]:
(
    pl_reservation
    .select("reserved_at")
    .with_columns(
        # （1） 30分を加算
        reserved_at_add30min=pl.col("reserved_at").dt.offset_by("30m"),
        # （2） 1時間を加算
        reserved_at_add1h=pl.col("reserved_at").dt.offset_by("1h"),
        # （3） 1日を加算
        reserved_at_add1d=pl.col("reserved_at").dt.offset_by("1d"),
        # （4） 1ヶ月を加算
        reserved_at_add1m=pl.col("reserved_at").dt.offset_by("1mo"),
        # （5） 1年を加算
        reserved_at_add1y=pl.col("reserved_at").dt.offset_by("1y"),
    )
)

### Q: 予約日時とチェックイン日の時間差を計算


#### Awesome

In [ ]:
(
    pl_reservation
    .select(["reserved_at", "checkin_date"])
    .with_columns(
        # （1） 年数差を計算
        diff_years=(
            (pl.col("checkin_date").dt.year()*12 + pl.col("checkin_date").dt.month())
            - (pl.col("reserved_at").dt.year()*12 + pl.col("reserved_at").dt.month())
            - pl.when(
                pl.col("checkin_date").dt.day() < pl.col("reserved_at").dt.day()
            ).then(1).otherwise(0)) // 12,
        # （2） 月数差を計算
        diff_months=
            (pl.col("checkin_date").dt.year()*12 + pl.col("checkin_date").dt.month())
            - (pl.col("reserved_at").dt.year()*12 + pl.col("reserved_at").dt.month())
            - pl.when(
                pl.col("checkin_date").dt.day() < pl.col("reserved_at").dt.day()
            ).then(1).otherwise(0),
        # （3） 日数差を計算
        diff_days=(pl.col("checkin_date") - pl.col("reserved_at")).dt.total_days(),
        # （4） 時間差を計算
        diff_hours=(pl.col("checkin_date") - pl.col("reserved_at")).dt.total_hours(),
        # （5） 分差を計算
        diff_minutes=(pl.col("checkin_date") - pl.col("reserved_at")).dt.total_minutes(),
        # （6） 秒差を計算
        diff_seconds=(pl.col("checkin_date") - pl.col("reserved_at")).dt.total_seconds()
    )
)

## 12-6 日時の数値化
### Q: チェックイン日の休日フラグの付与


#### Awesome

In [ ]:
from datetime import date
# （1） 日付マスタの作成
date_master = (
    # （1）-1 連続した日付の生成
    pl.DataFrame({"dt": pl.date_range(date(2014, 1, 1), date(2019, 12, 31),
        eager=True)})
    # （1）-2 祝日マスタの結合
    .join(
        pl_holiday.with_columns(holiday_date=pl.col("holiday_date").cast(pl.Date)),
        left_on="dt", right_on="holiday_date", how="left"
    )
    # （1）-3 休日フラグの作成
    .with_columns(is_day_off=pl.col("holiday_name").is_not_null()
            | pl.col("dt").dt.weekday().is_in([6, 7]))
    # （1）-4 次の日の休日フラグを取得して休日前日フラグを作成
    .with_columns(is_day_before_day_off=pl.col("is_day_off").shift(-1))
    .select(["dt", "is_day_off", "is_day_before_day_off"])
)

(
    pl_reservation
    # （2） checkin_dateを用いて日付マスタを結合
    .join(
        date_master.with_columns(dt=pl.col("dt").cast(pl.Datetime(time_unit="ns"))
                                    .dt.replace_time_zone("Asia/Tokyo")),
        left_on="checkin_date", right_on="dt", how="left"
    )
)